In [3]:
pip install streamlit pandas plotly

Note: you may need to restart the kernel to use updated packages.


In [8]:
import streamlit as st
import sqlite3
import json
from datetime import datetime

# Simple fix: Delete the database file to start fresh
import os

# Simple StudyTracker class
class StudyTracker:
    def __init__(self, db_path="progress.db"):
        self.db_path = db_path
        # Delete existing database to start fresh
        if os.path.exists(db_path):
            os.remove(db_path)
        self._init_db()
    
    def _init_db(self):
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''CREATE TABLE plans (
                id INTEGER PRIMARY KEY, name TEXT, created_date TEXT)''')
            conn.execute('''CREATE TABLE tasks (
                id INTEGER PRIMARY KEY, plan_id INTEGER, day_number INTEGER,
                description TEXT, est_mins INTEGER, is_completed BOOLEAN DEFAULT 0)''')
    
    def create_plan(self, name, json_data):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('INSERT INTO plans (name, created_date) VALUES (?, ?)',
                         (name, datetime.now().isoformat()))
            plan_id = cursor.lastrowid
            
            for day_data in json_data.get("days", []):
                for task in day_data["tasks"]:
                    cursor.execute('INSERT INTO tasks (plan_id, day_number, description, est_mins) VALUES (?, ?, ?, ?)',
                                 (plan_id, day_data["day"], task["description"], task.get("estimated_minutes", 30)))
            return plan_id
    
    def get_plans(self):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.execute('SELECT id, name FROM plans')
            return cursor.fetchall()
    
    def get_tasks(self, plan_id, day=None):
        with sqlite3.connect(self.db_path) as conn:
            if day:
                cursor = conn.execute('SELECT id, description, est_mins, is_completed FROM tasks WHERE plan_id = ? AND day_number = ?', (plan_id, day))
            else:
                cursor = conn.execute('SELECT id, day_number, description, est_mins, is_completed FROM tasks WHERE plan_id = ?', (plan_id,))
            return cursor.fetchall()
    
    def toggle_task(self, task_id):
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('UPDATE tasks SET is_completed = NOT is_completed WHERE id = ?', (task_id,))
    
    def get_progress(self, plan_id):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.execute('SELECT COUNT(*), SUM(is_completed) FROM tasks WHERE plan_id = ?', (plan_id,))
            total, completed = cursor.fetchone()
            return int((completed or 0) / total * 100) if total else 0

# Initialize
st.set_page_config(page_title="📚 Study Tracker", page_icon="📚")
if 'tracker' not in st.session_state:
    st.session_state.tracker = StudyTracker()

st.title("📚 Study Plan Tracker")

# Sidebar - Plan selection
with st.sidebar:
    plans = st.session_state.tracker.get_plans()
    
    if plans:
        plan_names = {f"{name} (ID: {pid})": pid for pid, name in plans}
        selected = st.selectbox("Choose Plan", list(plan_names.keys()))
        current_plan_id = plan_names[selected]
    else:
        st.write("No plans yet!")
        current_plan_id = None
    
    st.divider()
    
    # Create new plan
    st.subheader("Create New Plan")
    plan_name = st.text_input("Plan Name")
    
    sample_json = {
        "days": [
            {"day": 1, "tasks": [{"description": "Learn basics", "estimated_minutes": 30}]},
            {"day": 2, "tasks": [{"description": "Practice", "estimated_minutes": 45}]}
        ]
    }
    st.json(sample_json)
    
    json_input = st.text_area("JSON Plan", height=150)
    
    if st.button("Create Plan"):
        if plan_name and json_input:
            try:
                data = json.loads(json_input)
                st.session_state.tracker.create_plan(plan_name, data)
                st.success("Plan created!")
                st.rerun()
            except:
                st.error("Invalid JSON!")

# Main content
if current_plan_id:
    # Show progress
    progress = st.session_state.tracker.get_progress(current_plan_id)
    st.metric("Overall Progress", f"{progress}%")
    st.progress(progress / 100)
    
    # Day selector
    all_tasks = st.session_state.tracker.get_tasks(current_plan_id)
    days = sorted(set(task[1] for task in all_tasks))
    
    if days:
        selected_day = st.selectbox("Select Day", days, format_func=lambda x: f"Day {x}")
        
        st.subheader(f"📅 Day {selected_day} Tasks")
        
        # Show tasks for selected day
        day_tasks = st.session_state.tracker.get_tasks(current_plan_id, selected_day)
        
        for task_id, desc, mins, completed in day_tasks:
            col1, col2, col3 = st.columns([0.5, 3, 1])
            
            with col1:
                # Store previous state to detect changes
                checkbox_key = f"task_{task_id}"
                current_checked = st.checkbox("", value=bool(completed), key=checkbox_key)
                
                # Only toggle if state actually changed
                if current_checked != bool(completed):
                    st.session_state.tracker.toggle_task(task_id)
                    st.rerun()
            
            with col2:
                if completed:
                    st.write(f"~~{desc}~~ ✅")
                else:
                    st.write(f"**{desc}**")
            
            with col3:
                st.caption(f"⏱️ {mins} min")

else:
    st.info("Create your first study plan using the sidebar!")
    
    # Quick demo button
    if st.button("🚀 Create Demo Plan", type="primary"):
        demo = {
            "days": [
                {"day": 1, "tasks": [
                    {"description": "Learn Python basics", "estimated_minutes": 45},
                    {"description": "Practice variables", "estimated_minutes": 30}
                ]},
                {"day": 2, "tasks": [
                    {"description": "Study loops", "estimated_minutes": 40}
                ]}
            ]
        }
        st.session_state.tracker.create_plan("Python Demo", demo)
        st.rerun()

2025-08-30 23:54:29.686 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-30 23:54:29.687 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'progress.db'